# Défi IA 2023 
## Bedbugs - INSA 5 MA
## Lila, Léa & Flavie

# Code de requêtes


In [1]:
# librairies
# --------------------------------------------
from datetime import datetime
import pandas
import numpy as np
import json
import csv
import time
import requests
import pandas as pd 
import urllib.parse

In [2]:
# connexions à l'api
# --------------------------------------------
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

user_id = 'f80b400f-3d29-43f0-8049-214a756ff0b3' # notre user ID reçu par mail

### Création d'un dataset des requêtes possibles (hors variation des jours (date))
- Stockage dans un fichier csv : allrequest.csv

<font color = 'red'>ok c'est fait ! Il faut rajouter la ville 'belgian' dans la lst2 et recontruire notre dataframe `allresquest` </font>


In [3]:
'''lst1=[0,1]   
lst2=['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish','swedish']
lst3=['amsterdam', 'copenhagen', 'madrid','paris', 'rome', 'sofia', 'valletta', 'vienna', 'vilnius']
print(len(lst2))
lst = [[[j,k,l] for j in lst1] for k in lst2 for l in lst3]

A=np.array(lst)
A=np.reshape(A, (-1,3))
all_requests=pandas.DataFrame(A, columns=["mobile", "language", "city"])
all_requests
all_requests.to_csv('allrequest.csv', index=False)
'''

27


In [3]:
#à faire lors de la première utilisation 
#1) lecture du csv 
df = pandas.read_csv('allrequest.csv', header = 0)
df

,mobile,language,city
0,0,austrian,amsterdam
1,1,austrian,amsterdam
2,0,austrian,copenhagen
3,1,austrian,copenhagen
4,0,austrian,madrid
...,...,...,...
481,1,swedish,valletta
482,0,swedish,vienna
483,1,swedish,vienna
484,0,swedish,vilnius


### Création du fichier de nos requêtes effectuées (pas leur résultats)
- stockage de la liste des requêtes dans un fichier all_our_requests_done.csv

In [4]:
df

,mobile,language,city
0,0,austrian,amsterdam
1,1,austrian,amsterdam
2,0,austrian,copenhagen
3,1,austrian,copenhagen
4,0,austrian,madrid
...,...,...,...
481,1,swedish,valletta
482,0,swedish,vienna
483,1,swedish,vienna
484,0,swedish,vilnius


In [6]:
### attention si recompilation : le fichier existant sera écrasé !! 
"""all_our_requests=pandas.DataFrame(columns=["avatar_id", "mobile", "language", "city", "date", "id_request"])
all_our_requests.to_csv('all_our_requests_done.csv', index=False) """

'all_our_requests=pandas.DataFrame(columns=["avatar_id", "mobile", "language", "city", "date", "id_request"])\nall_our_requests.to_csv(\'all_our_requests_done.csv\', index=False) '

### Création du fichier des résultats des requêtes effectuées
- stockage de la liste des requêtes dans un fichier pricing_requests_done.csv

In [7]:
#création du fichier des résultats des requêtes
### attention si recompilation : le fichier existant sera écrasé !! 
"""pricing_requests=pandas.DataFrame(columns=["hotel_id", "price", "stock", "city", "date", "language", "mobile", "avatar_id", "request_number"])"""
"""pricing_requests.to_csv('pricing_requests_done.csv', index=False)"""

"pricing_requests.to_csv('pricing_requests_done.csv', index=False)"

array([14, 15])

### Bloc de requête à l'api et stockage des données avec les précédentes

<font color = "blue"> **Questions (Lila):** 
- <font color = "blue"> Dans la commande `chosen_idx = np.random.choice(idx_available, replace = False, size = nb_request)`, on tire aléatoirement sans remise len(df) = 468 éléments parmi une liste de taille 468: [0,1,...,467] ce qui revient exactement à choisir la liste [0,1,...,467] directement (sans avoir besoin du random.choice). Est-ce voulu ?

- <font color = "blue"> `our_requests = pandas.read_csv('all_our_requests_done.csv', header = 0)` au début, ce tableau est vide non ? Pour la première utilisation, il s'agit de `allrequest.csv` ? J'imagine qu'à l'origine il devait y avoir marqué `allrequest.csv` et non pas `all_our_requests_done.csv`.
    
- <font color = "blue"> On a `days= np.random.choice(selected_days, number_days, replace=False)` et `days= np.random.choice(range(0,45), number_days, replace=False)` à la suite donc seule la 2e version est prise en compte. 
    
- <font color = "blue"> On crée un "avatar_name" avec la date du jour mais après je n'ai pas l'impression qu'on stocke cette info quelque part. Une fois le tableau constitué, on ne peut pas retrouver cette info. 
    
- <font color = "blue"> l'Id de a request est i dans la ligne `add_to_our_requests = ...` mais du coup, si on fait tourner plusieurs fois le même code, on n'aura pas des requêtes avec le même id ? comment on va faire pour les différencier ? Il ne faudrait pas mettre `request_num` à la place de `i` ?

In [17]:
print(df.iloc[35,:])

mobile            1
language    belgian
city        vilnius
Name: 35, dtype: object


In [6]:
#variables 
number_days = 1 # nombre de jour pour lesquels un utilisateur va faire la même requête

#choix du ou des indexes dans la table des requêtes :
idx_available = range(0,len(df)) # len(df) = 486
#nombre de requête
nb_request = len(df) # = 486
#tirage sans remise de "nb_request" élements dans la liste "idx_available = [0,1,...,len(df)]"
chosen_idx = np.random.choice(idx_available, replace = False, size = nb_request)

#chargement de toutes les requêtes possibles et de nos données existantes
our_requests = pandas.read_csv('all_our_requests_done.csv', header = 0)
pricing_requests = pandas.read_csv('pricing_requests_done.csv', header = 0)

#variable qui sert à numéroter nos requêtes (utile si plusieurs requêtes sont faites plusieurs fois)
request_num = our_requests.shape[0]

for i in chosen_idx: #le chosen index correspond à une requête dans our_requests #### ou plutôt allrequest.csv ?
    print(i)
    date = datetime.today().strftime('%Y-%m-%d_%H:%M:%S')
    
    selected_daysA = [0,1,2,3,4,5,6,7]
    selected_daysB = [13,14,15,16,17,18,19,20,21,22]
    selected_daysC = [32,33,34,35,36,37,38,39]
    b=[0,1]
    #tirage sans remise de "number_days" éléments dans la liste "selected_days"
    #days = np.random.choice(b, number_days, replace=False)
    

    #days= np.random.choice(selected_daysC, number_days, replace=False) 
    days= np.random.choice(range(0,45), number_days, replace=False)
    j=0
    
     
    #cas 1 avatar fait la même requête sur number_days différents jours
    #création avatar
    time.sleep(1) #pour ne pas avoir 2 avatar de noms identiques
    name = date + '_avatar'     
    #à créer l'avatar 
    a = requests.post(path(f'avatars/{user_id}/{name}'))
    
    for iter_days in (sorted(days, reverse=True)):
        
        #paramètres requête de our_requests #### de df=allrequest plutôt ? 
        city=str(df.iloc[i,:]["city"])
        mobile=df.iloc[i,:]["mobile"]
        language=str(df.iloc[i,:]["language"])
        
        #paramètres requetes
        params = {
            "avatar_name": name,
            "language": language,
            "city": city,
            "date": iter_days,
            "mobile": mobile,
        }
        
        print(city)
        print(language)
        
        request_num +=1
        #requête à l'api
        r1 = requests.get(path(f"pricing/{user_id}"), params=params)
        print(r1)
        #mise en forme 
        array_to_add = pd.DataFrame(r1.json()['prices']).assign(**r1.json()['request'])
        array_to_add = array_to_add.assign(request_number=request_num)
       
        
        pricing_requests = pd.concat([pricing_requests, array_to_add])
        
        add_to_our_requests = pandas.DataFrame([[r1.json()['request']['avatar_id'], mobile, language, city, iter_days,  i]], 
                                       columns=["avatar_id", "mobile", "language", "city", "date", "id_request"])
        our_requests = pandas.concat([our_requests, add_to_our_requests], ignore_index=True)
        
#enregistrement
pricing_requests.to_csv('pricing_requests_done.csv', index=False)
our_requests.to_csv('all_our_requests_done.csv', index=False)

459
rome
spanish
<Response [200]>
204
paris
german
<Response [200]>
280
sofia
italian
<Response [200]>
482
vienna
swedish
<Response [200]>
1
amsterdam
austrian
<Response [200]>
48
valletta
bulgarian
<Response [200]>
348
paris
maltese
<Response [200]>
111
copenhagen
danish
<Response [200]>
362
copenhagen
polish
<Response [200]>
358
vilnius
maltese
<Response [200]>
334
sofia
luxembourgish
<Response [200]>
447
vienna
slovene
<Response [200]>
50
vienna
bulgarian
<Response [200]>
322
vilnius
lithuanian
<Response [200]>
421
paris
slovakian
<Response [200]>
317
sofia
lithuanian
<Response [200]>
377
vilnius
polish
<Response [200]>
44
rome
bulgarian
<Response [200]>
400
madrid
romanian
<Response [200]>
272
copenhagen
italian
<Response [200]>
184
madrid
french
<Response [200]>
151
paris
estonian
<Response [200]>
86
vienna
cypriot
<Response [200]>
467
vilnius
spanish
<Response [200]>
331
paris
luxembourgish
<Response [200]>
68
vienna
croatian
<Response [200]>
356
vienna
maltese
<Response [200]>
1

In [19]:
'''#pour quand on s'est raté et que l'enregistrement rate
pricing_requests.to_csv('pricing_requests_done.csv', index=False)
our_requests.to_csv('all_our_requests_done.csv', index=False)'''

## Stockage des paires noms_utilisateur, identifiants (à mettre à jour souvent en exécutant cette cellule)
Il suffira d'aller dans cette table pour à partir de l'identifiant id retrouver le nom de l'utilisateur

In [10]:
'''r = requests.get(path(f"avatars/{user_id}"))
tab=[]
for avatar in r.json():
    tab.append([avatar['id'], avatar['name']])

df = pd.DataFrame(tab)
df.columns = ['avatar_Id', 'avatar_Name']
df.to_csv('tab_id_name_of_users.csv', index=False)'''

'r = requests.get(path(f"avatars/{user_id}"))\ntab=[]\nfor avatar in r.json():\n    tab.append([avatar[\'id\'], avatar[\'name\']])\n\ndf = pd.DataFrame(tab)\ndf.columns = [\'avatar_Id\', \'avatar_Name\']\ndf.to_csv(\'tab_id_name_of_users.csv\', index=False)'